In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

# やりたいこと
+ With check optionのユースケースをまとめる

## with check optionとは
+ insert/update時にもwhere句の条件をチェックさせるオプション
  + check制約では相関サブクエリを入れられない代わりにwith check optionが使える

In [4]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    value boolean not null
);

insert into tbl
values (1,true),
    (2,true),
    (3,false),
    (4,false);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
4 rows affected.


[]

In [9]:
%%sql
drop view if exists v1;
create view v1 as
select *
from tbl
where value = true
with check option;

-- insert, updateをしようとすると失敗する
-- 新しい行はビュー"v1"のチェックオプションに違反しています
insert into v1
values (5,false);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
(psycopg2.errors.WithCheckOptionViolation) 新しい行はビュー"v1"のチェックオプションに違反しています
DETAIL:  失敗した行は(5, f)を含みます

[SQL: -- insert, updateをしようとすると失敗する
-- 新しい行はビュー"v1"のチェックオプションに違反しています
insert into v1
values (5,false);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [11]:
%%sql
-- 新しい行はビュー"v1"のチェックオプションに違反しています
update v1
set value = false
where id = 1;

*  postgresql://padawan:***@db:5432/dsdojo_db
(psycopg2.errors.WithCheckOptionViolation) 新しい行はビュー"v1"のチェックオプションに違反しています
DETAIL:  失敗した行は(1, f)を含みます

[SQL: update v1
set value = false
where id = 1;]
(Background on this error at: https://sqlalche.me/e/14/f405)


## 設定した範囲がお互い重複しないようにする
+ 相関条件(相関サブクエリ)が必要なので、check制約では記述できない

In [12]:
%%sql
-- sql puzzle#47
drop view if exists CheckReserved;
drop table if exists Reservations;
create table Reservations (
    reserver char(10) not null primary key,
    start_seat integer not null,
    finish_seat integer not null
);

insert into Reservations
values ('Eenie', 1,4),
    ('Meanie', 6,7),
    ('Mynie', 10, 15),
    ('Melvin', 16, 18);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
4 rows affected.


[]

In [13]:
%%sql
create view CheckReserved as
select reserver, start_seat, finish_seat
from Reservations as r1
-- with check optionでwhere句がチェックされる
where not exists (
    select *
    from Reservations as r2
    where r1.reserver <> r2.reserver and
    (r1.start_seat <= r2.finish_seat and r2.start_seat <= r1.finish_seat)
)
with check option;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.


[]

In [14]:
%%sql
-- 範囲が被るので、チェックオプションに違反する
insert into CheckReserved
values ('dummy', 5, 8);

*  postgresql://padawan:***@db:5432/dsdojo_db
(psycopg2.errors.WithCheckOptionViolation) 新しい行はビュー"checkreserved"のチェックオプションに違反しています
DETAIL:  失敗した行は(dummy     , 5, 8)を含みます

[SQL: -- 範囲が被るので、チェックオプションに違反する
insert into CheckReserved
values ('dummy', 5, 8);]
(Background on this error at: https://sqlalche.me/e/14/f405)
